In [1]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.85
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [58]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [59]:
img_height, img_width = (224,224)
batch_size = 32
train_data_dir="./processed_data/train"
valid_data_dir="./processed_data/val"
test_data_dir="./processed_data/test"
IMAGE_SIZE = [224, 224]

In [60]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_height,img_width),
                                                   batch_size=batch_size,
                                                    color_mode="rgb",
                                                   class_mode='categorical'
                                                   )
test_datagen = ImageDataGenerator(rescale = 1./255)
valid_generator = test_datagen.flow_from_directory(valid_data_dir,
                                                   target_size=(img_height,img_width),
                                                   color_mode="rgb",
                                                   batch_size=batch_size,
                                                   class_mode='categorical'
                                                   )

Found 2880 images belonging to 3 classes.
Found 720 images belonging to 3 classes.


In [61]:
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                   target_size=(img_height,img_width),
                                                   batch_size=1,
                                                  color_mode="rgb",
                                                   class_mode='categorical')

Found 162 images belonging to 3 classes.


In [64]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,3,3,padding='same',input_shape=(224,224,3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(96,3,3,padding='valid'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    
        

    tf.keras.layers.Conv2D(128,3,3,padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(256,3,3,padding='valid'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.ZeroPadding2D(padding=(1,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Dropout(0.5),
    
    
   
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3),
    tf.keras.layers.Activation('softmax')
    
       
])

In [65]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [67]:
history = model.fit(train_generator,
                    epochs=30,
                    validation_data = valid_generator,
                   )

Epoch 1/30
90/90 [==============================] - 90s 1s/step - loss: 0.9724 - accuracy: 0.5295 - val_loss: 0.6150 - val_accuracy: 0.6375
Epoch 2/30
90/90 [==============================] - 92s 1s/step - loss: 0.7366 - accuracy: 0.6323 - val_loss: 0.5669 - val_accuracy: 0.6542
Epoch 3/30
90/90 [==============================] - 107s 1s/step - loss: 0.6682 - accuracy: 0.6955 - val_loss: 0.4895 - val_accuracy: 0.8000
Epoch 4/30
90/90 [==============================] - 107s 1s/step - loss: 0.6011 - accuracy: 0.7462 - val_loss: 0.3954 - val_accuracy: 0.8597
Epoch 5/30
90/90 [==============================] - 108s 1s/step - loss: 0.5668 - accuracy: 0.7691 - val_loss: 0.3781 - val_accuracy: 0.8833
Epoch 6/30
90/90 [==============================] - 108s 1s/step - loss: 0.5297 - accuracy: 0.7833 - val_loss: 0.3225 - val_accuracy: 0.8750
Epoch 7/30
90/90 [==============================] - 103s 1s/step - loss: 0.4822 - accuracy: 0.8052 - val_loss: 0.3137 - val_accuracy: 0.8833
Epoch 8/30
90/9

In [137]:
from tensorflow.keras.models import load_model

model.save('custom_model.h5')